In [47]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [48]:
import warnings
warnings.filterwarnings("ignore")

In [49]:
import os
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
from shutil import copyfile

In [50]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.callbacks.hooks import params_size

In [51]:
path = Path('D:/Datasets/EIPH WSI/Pferd/Patches/train')
files = {p.name:p for p in Path("D:\\Datasets\\EIPH WSI\\Pferd").glob("**/*.svs")}

learn = load_learner(path)

configuration = Configuration()
configuration.username = 'marzahl'
configuration.password = '****'
configuration.host =  "https://exact.cs.fau.de/"


# Sync results with server

In [32]:
!pip install --upgrade EXCAT-Sync

Requirement already up-to-date: EXCAT-Sync in d:\progprojekte\python\promotion\exact-sync (0.0.13)


In [33]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi
from exact_sync.v1.api.teams_api import TeamsApi

from exact_sync.v1.models import ImageSet, Team, Product, AnnotationType, Image, Annotation, AnnotationMediaFile
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [34]:
client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [35]:
image_sets = image_sets_api.list_image_sets(name__contains="-Algorithm")
image_sets.results

[{'creator': 1,
  'description': '',
  'id': 75,
  'images': [1126,
             1127,
             1128,
             1129,
             1130,
             1131,
             1132,
             1133,
             1134,
             1135,
             1136,
             1137,
             1138,
             1139,
             1140,
             1141,
             1142,
             1143],
  'location': None,
  'main_annotation_type': 24,
  'name': 'BerlinerBlau-Algorithm',
  'path': 'exact_2_75',
  'product_set': [6],
  'set_tags': [],
  'team': 2}, {'creator': 1,
  'description': '',
  'id': 76,
  'images': [1144,
             1145,
             1146,
             1147,
             1148,
             1149,
             1150,
             1151,
             1152,
             1153,
             1154,
             1155,
             1156,
             1157,
             1158,
             1159,
             1160,
             1161,
             1162,
             1163,
             116

In [40]:
import openslide
import cv2
from torchvision import transforms

In [46]:
down_factor = 1
level = 0

mean, std = data.stats

device = torch.device('cpu')
cpu_model = learn.model.to(device)
with torch.no_grad():
    for image_set in image_sets.results:
        for image_id in tqdm(image_set.images):
            image = images_api.retrieve_image(id=image_id)

            slide_path = files[image.name]

            slide = openslide.open_slide(str(slide_path))

            for anno in annotations_api.list_annotations(image=image_id, pagination=False, meta_data__isnull=True).results:
                if anno.vector == None:
                    continue
                
                vector = anno.vector
                offset_x = int(vector["x1"])
                offset_y = int(vector["y1"])
                w = int(vector["x2"]) - int(vector["x1"])
                h = int(vector["y2"]) - int(vector["y1"])

                if w < 15 or h < 15:
                    continue

                patch = np.array(slide.read_region(location=(int(offset_x * down_factor), int(offset_y * down_factor)),
                                                level=level, size=(w, h)))[:, :, :3]

                patch = cv2.resize(patch, (128, 128))
                patch = pil2tensor(patch / 255., np.float32)
                patch = transforms.Normalize(mean, std)(patch)

                score = cpu_model.eval()(patch[None, :, :, :])
        
                meta = {'Score': float(score)}

                annotations_api.partial_update_annotation(id=anno.id, meta_data=meta)








  0%|                                                                                           | 0/18 [00:00<?, ?it/s]






  6%|████▌                                                                              | 1/18 [00:01<00:18,  1.06s/it]






 11%|█████████▏                                                                         | 2/18 [00:01<00:13,  1.15it/s]






 17%|█████████████▊                                                                     | 3/18 [00:01<00:10,  1.37it/s]






 22%|██████████████████▍                                                                | 4/18 [00:02<00:11,  1.21it/s]






 28%|███████████████████████                                                            | 5/18 [00:03<00:09,  1.32it/s]






 33%|███████████████████████████▋                                                       | 6/18 [00:04<00:08,  1.38it/s]






 39%|████████████████████████████████▎                                                  | 7/18 [00:05<00:08,  1